In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment/sentiment.csv


In [14]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')  

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

# Sentiment140 dataset with 1.6 million tweets
This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment.

dataset from: https://www.kaggle.com/datasets/kazanova/sentiment140


# Loading Dataset

In [3]:
sentiment_df = pd.read_csv('/kaggle/input/sentiment/sentiment.csv', encoding='ISO-8859-1')
sentiment_df

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [4]:
sentiment_df.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

In [5]:
# New column names
sentiment_df = sentiment_df.rename(columns={
    '0': 'target',
    '@switchfoot http://twitpic.com/2y1zl - Awww, that\'s a bummer.  You shoulda got David Carr of Third Day to do it. ;D': 'text'
})

In [6]:
sentiment_df = sentiment_df[['target', 'text']]
sentiment_df

sentiment_mapping = {0: 'negative', 2: 'neutral', 4: 'positive'}
sentiment_df['sentiment'] = sentiment_df['target'].map(sentiment_mapping)
sentiment_df

,target,text,sentiment
0,0,is upset that he can't update his Facebook by ...,negative
1,0,@Kenichan I dived many times for the ball. Man...,negative
2,0,my whole body feels itchy and like its on fire,negative
3,0,"@nationwideclass no, it's not behaving at all....",negative
4,0,@Kwesidei not the whole crew,negative
...,...,...,...
1599994,4,Just woke up. Having no school is the best fee...,positive
1599995,4,TheWDB.com - Very cool to hear old Walt interv...,positive
1599996,4,Are you ready for your MoJo Makeover? Ask me f...,positive
1599997,4,Happy 38th Birthday to my boo of alll time!!! ...,positive


In [7]:
numeric_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
sentiment_df['Sentiment_Numeric'] = sentiment_df['sentiment'].map(numeric_mapping)
sentiment_df = sentiment_df.drop(columns=['target'])
sentiment_df


,text,sentiment,Sentiment_Numeric
0,is upset that he can't update his Facebook by ...,negative,0
1,@Kenichan I dived many times for the ball. Man...,negative,0
2,my whole body feels itchy and like its on fire,negative,0
3,"@nationwideclass no, it's not behaving at all....",negative,0
4,@Kwesidei not the whole crew,negative,0
...,...,...,...
1599994,Just woke up. Having no school is the best fee...,positive,2
1599995,TheWDB.com - Very cool to hear old Walt interv...,positive,2
1599996,Are you ready for your MoJo Makeover? Ask me f...,positive,2
1599997,Happy 38th Birthday to my boo of alll time!!! ...,positive,2


# Clean-up Dataset

In [8]:
def remove_non_text(content):
    content = re.sub(r"http\S+|www\S+|https\S+", '', content, flags=re.MULTILINE)  # delete the URL
    content = re.sub(r'\[.*?\]', '', content)  # delete the content in square brackets
    content = re.sub(r'[^A-Za-z\s]', '', content)  # keep only letters and spaces, delete other characters
    return content

sentiment_df['cleaned_text'] = sentiment_df['text'].apply(remove_non_text)

sentiment_df = sentiment_df.drop_duplicates(subset='cleaned_text')

sentiment_df = sentiment_df.dropna(subset=['cleaned_text'])
sentiment_df = sentiment_df[sentiment_df['cleaned_text'].str.strip() != '']
sentiment_df['cleaned_text'] = sentiment_df['cleaned_text'].str.lower().str.strip()
sentiment_df

,text,sentiment,Sentiment_Numeric,cleaned_text
0,is upset that he can't update his Facebook by ...,negative,0,is upset that he cant update his facebook by t...
1,@Kenichan I dived many times for the ball. Man...,negative,0,kenichan i dived many times for the ball manag...
2,my whole body feels itchy and like its on fire,negative,0,my whole body feels itchy and like its on fire
3,"@nationwideclass no, it's not behaving at all....",negative,0,nationwideclass no its not behaving at all im ...
4,@Kwesidei not the whole crew,negative,0,kwesidei not the whole crew
...,...,...,...,...
1599994,Just woke up. Having no school is the best fee...,positive,2,just woke up having no school is the best feel...
1599995,TheWDB.com - Very cool to hear old Walt interv...,positive,2,thewdbcom very cool to hear old walt interviews
1599996,Are you ready for your MoJo Makeover? Ask me f...,positive,2,are you ready for your mojo makeover ask me fo...
1599997,Happy 38th Birthday to my boo of alll time!!! ...,positive,2,happy th birthday to my boo of alll time tupac...


In [9]:
!pip install tqdm

In [10]:
from tqdm import tqdm

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

# initialize tqdm progress bar
tqdm.pandas()

# use progress_apply to process data and show progress bar
sentiment_df['cleaned_text'] = sentiment_df['cleaned_text'].progress_apply(remove_punctuation)

100%|██████████| 1574711/1574711 [00:08<00:00, 180575.82it/s]


In [11]:
sentiment_df

,text,sentiment,Sentiment_Numeric,cleaned_text
0,is upset that he can't update his Facebook by ...,negative,0,is upset that he cant update his facebook by t...
1,@Kenichan I dived many times for the ball. Man...,negative,0,kenichan i dived many times for the ball manag...
2,my whole body feels itchy and like its on fire,negative,0,my whole body feels itchy and like its on fire
3,"@nationwideclass no, it's not behaving at all....",negative,0,nationwideclass no its not behaving at all im ...
4,@Kwesidei not the whole crew,negative,0,kwesidei not the whole crew
...,...,...,...,...
1599994,Just woke up. Having no school is the best fee...,positive,2,just woke up having no school is the best feel...
1599995,TheWDB.com - Very cool to hear old Walt interv...,positive,2,thewdbcom very cool to hear old walt interviews
1599996,Are you ready for your MoJo Makeover? Ask me f...,positive,2,are you ready for your mojo makeover ask me fo...
1599997,Happy 38th Birthday to my boo of alll time!!! ...,positive,2,happy th birthday to my boo of alll time tupac...
